In [1]:
import os
import pandas as pd
import re

In [2]:
from google.cloud import bigquery

In [3]:
project_id = 'mimic-371816'

In [4]:
BigQuery_client = bigquery.Client(project=project_id)

In [5]:
# This is a practice query to ensure you can pull a short query from mimic
practice = '''SELECT * FROM `lcp-consortium.mimiciv_note.radiology` LIMIT 10;'''

In [6]:
# Put the short practice query into a dataframe
dfpractice = BigQuery_client.query(practice).to_dataframe()

In [7]:
dfpractice.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,17693798-RR-256,17693798,26855677,RR,256,2125-05-20 21:00:00,2125-05-21 10:03:00,CLINICAL HISTORY: Patient fell onto right rib...
1,12165964-RR-256,12165964,29176015,RR,256,2152-07-01 03:52:00,2152-07-01 12:57:00,AP CHEST 4:52 A.M. ON ___\n\nHISTORY: CHF and...
2,17329106-RR-256,17329106,23935599,RR,256,2158-08-20 19:36:00,2158-08-20 21:16:00,HISTORY: Altered mental status.\n\nCOMPARISON...
3,16921793-RR-256,16921793,23285822,RR,256,2134-05-01 07:12:00,2134-05-01 08:57:00,INDICATION: History of end-stage renal diseas...
4,14867101-RR-257,14867101,24955503,RR,257,2151-01-28 18:42:00,2151-01-28 19:52:00,CHEST RADIOGRAPH\n\nComparison is made with a ...


In [8]:
# A SQL query that pulls all radiology reports that have an ICD code related to PE
Reports = '''--CTE for diagnosis table
WITH dx AS
(SELECT subject_id AS dx_MRN, hadm_id AS dx_hadm, icd_code AS dx_icdcode
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`),

--CTE for radiology table
reports AS 
(SELECT subject_id AS report_MRN, note_id, hadm_id AS report_hadm, charttime, text, note_type
FROM `lcp-consortium.mimiciv_note.radiology`)

SELECT * from reports
LEFT JOIN dx
ON reports.report_MRN = dx.dx_MRN
WHERE 
(dx.dx_icdcode LIKE 'I26%'
OR dx.dx_icdcode LIKE '415%')
AND note_type = 'RR';
'''

In [9]:
reports = BigQuery_client.query(Reports).to_dataframe()

In [10]:
len(reports)

147400

In [16]:
# Now look for the following PE phrases in the text column
PEphrases = ['pulmonary embolism', 'pulmonary emboli', 'pulmonary embolus', 'pulmonary embolic', 'pulmonary thromboembolism', 'pulmonary thromboembolic', 'pulmonary artery embolus', 'pulmonary artery emboli', 'pulmonary artery embolism', 'pulmonary artery embolic', 'pulmonary artery thromboembolism', 'pulmonary artery thromboembolic', 'pulmonary arterial embolus', 'pulmonary arterial emboli', 'pulmonary arterial embolism', 'pulmonary arterial embolic', 'pulmonary arterial thromboembolism', 'pulmonary arterial thromboembolic']
# Create a regular expression pattern that matches any of the phrases
PEpattern = re.compile('|'.join(PEphrases), flags=re.IGNORECASE)

In [17]:
ReportswithPE = reports[reports['text'].str.contains(PEpattern)]

In [18]:
len(ReportswithPE)

11371

In [20]:
## A selection of 100 reports from ReportswithPE is assessed to determine the RegEx pattern
# for identifying CTA reports

In [21]:
# Now, a SQL query that will pull all radiology reports
RR = '''SELECT * FROM `lcp-consortium.mimiciv_note.radiology`;
'''

In [22]:
RR = BigQuery_client.query(RR).to_dataframe()

In [47]:
len(RR)

2321355

In [23]:
# Segment the radiology reports into PROCEDURE, EXAMINATION, STUDY, and TECHNIQUE
procedure_pattern = re.compile(r'PROCEDURE:(.*?)(?=\n\n[A-Z]+:|\n\n|\n$)', re.DOTALL)
examination_pattern = re.compile(r'EXAMINATION:(.*?)(?=\n\n[A-Z]+:|\n\n|\n$)', re.DOTALL)
study_pattern = re.compile(r'STUDY:(.*?)(?=\n\n[A-Z]+:|\n\n|\n$)', re.DOTALL)
technique_pattern = re.compile(r'TECHNIQUE:(.*?)(?=\n\n[A-Z]+:|\n\n|\n$)', re.DOTALL)

In [24]:
# Create a function to extract both EXAMINATION and STUDY and TECHNIQUE from the text column
def extract_exam(text):
    examination_match = examination_pattern.search(text)
    study_match = study_pattern.search(text)
    technique_match = technique_pattern.search(text)
    procedure_match = procedure_pattern.search(text)
    
    examination = examination_match.group(1).strip() if examination_match and examination_match.group(1) else None
    study = study_match.group(1).strip() if study_match and study_match.group(1) else None
    procedure = procedure_match.group(1).strip() if procedure_match and procedure_match.group(1) else None
    technique = technique_match.group(1).strip() if technique_match and technique_match.group(1) else None
    
    return examination, study, procedure, technique

In [25]:
# Let's try RegEx on the full dataset now and see how many CTA chests I get
RR[['Examination', 'Study', 'Procedure', 'Technique']] = RR['text'].apply(extract_exam).apply(pd.Series)

In [26]:
# Now try the string contains search approach
terms = ['cta chest', 'cta thorax', 'ct pulmonary angiogram', 'chest cta', 'ctpa', 'cta protocol', 'cta of the chest' 'torso cta', 'cta torso', 'ct pa protocol']
RR['cta_chest_string'] = RR['Examination'].str.lower().str.contains('|'.join(terms)) | \
                  RR['Procedure'].str.lower().str.contains('|'.join(terms)) | \
                  RR['Study'].str.lower().str.contains('|'.join(terms)) | \
                  RR['Technique'].str.lower().str.contains('|'.join(terms))

In [39]:
# This dataframe will only include rows where cta chest evaluated to true 
CTAchest = RR[RR['cta_chest_string'] == True]

In [28]:
len(CTAchest)

17143

In [29]:
# Make a copy to work with
CTAchestcopy = CTAchest.copy()

In [30]:
# Remove all of the notes that I identified as CTA chest
RRremaining = RR[~RR['note_id'].isin(CTAchest['note_id'])]

In [32]:
# Define three regex patterns for isolated CTA Chest sections
ctapatternone = re.compile(r'\n\nCTA CHEST\b')
ctapatterntwo = re.compile(r'\n\nCTA OF THE CHEST\b')
ctapatternthree = re.compile(r'\n\nCTA THORAX\b')

In [34]:
# Define patterns
patterns = [ctapatternone, ctapatterntwo, ctapatternthree]

In [35]:
def find_cta_chest_section(text, patterns):
    for pattern in patterns:
        if pattern.search(text):
            return True
    return False

In [37]:
# Create a copy of the RRremaining dataframe
RRremainingCTAsection = RRremaining.copy()

In [38]:
# Apply the function to create a new column
RRremainingCTAsection['cta_chest_section'] = RRremainingCTAsection['text'].apply(lambda x: find_cta_chest_section(x, patterns))

In [41]:
# This dataframe will only include rows where cta chest SECTION evaluated to true 
CTAsection = RRremainingCTAsection[RRremainingCTAsection['cta_chest_section'] == True]

In [42]:
len(CTAsection)

2707

In [43]:
# Concactenate CTAchest and CTAsection
CTAchestworking = pd.concat([CTAchest, CTAsection], axis=0)

In [45]:
# Remove notes found in CTAchestworking from the original full radiology report dataframe
RRfinalremaining = RR[~RR['note_id'].isin(CTAchestworking['note_id'])]

In [46]:
len(RRfinalremaining)

2301505

In [48]:
# Let's segment to history and indication
history_pattern = re.compile(r'HISTORY:(.*?)(?=\n\n[A-Z]+:|\n\n|\n$)', re.DOTALL)
indication_pattern = re.compile(r'INDICATION:(.*?)(?=\n\n[A-Z]+:|\n\n|\n$)', re.DOTALL)

In [49]:
# Create a function to extract both HISTORY and INDICATION from the text column
def extract_history(text):
    history_match = history_pattern.search(text)
    indication_match = indication_pattern.search(text)
    
    history = history_match.group(1).strip() if history_match and history_match.group(1) else None
    indication = indication_match.group(1).strip() if indication_match and indication_match.group(1) else None
 
    return history, indication

In [50]:
# Make a copy of the RRfinalremaining dataframe
RRwithPE = RRfinalremaining.copy()

In [51]:
# Let's apply to above function to extract history and indication
RRwithPE[['History', 'Indication']] = RRwithPE['text'].apply(extract_history).apply(pd.Series)

In [52]:
# Now try the string contains search approach
terms = ['pulmonary embolism', 'pulmonary emboli', 'pulmonary embolus', 'pulmonary embolic', 'pulmonary thromboembolism', 'pulmonary thromboembolic', 'pulmonary artery embolus', 'pulmonary artery emboli', 'pulmonary artery embolism', 'pulmonary artery embolic', 'pulmonary artery thromboembolism', 'pulmonary artery thromboembolic', 'pulmonary arterial embolus', 'pulmonary arterial emboli', 'pulmonary arterial embolism', 'pulmonary arterial embolic', 'pulmonary arterial thromboembolism', 'pulmonary arterial thromboembolic']
RRwithPE['pe_string'] = RRwithPE['History'].str.lower().str.contains('|'.join(terms)) | \
                  RRwithPE['Indication'].str.lower().str.contains('|'.join(terms))

In [54]:
# This dataframe will only include rows where pe_string evaluates to true
RRwithPEtrue = RRwithPE[RRwithPE['pe_string'] == True]

In [55]:
len(RRwithPEtrue)

2077

In [56]:
# Concactenate the CTAchestworking dataframe with the RRwithPEtrue dataframe
CTAfinal = pd.concat([CTAchestworking, RRwithPEtrue], axis=0)

In [57]:
len(CTAfinal)

21927